# ETL Pipeline 
按照下面的说明，创建你的 ETL 管道。
### 导入与加载.
- 导入 Python 库
- 将 `messages.csv` 导入数据框，查看前几行
- 将 `categories.csv` 导入数据框，查看前几行

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('..\\data\\disaster_messages.csv',dtype=str)
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('..\\data\\disaster_categories.csv',dtype=str)
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 合并数据集
- 使用相同的 id 合并 messages 和 categories 数据集
- 将合并后的数据集传入 `df`，在后续的步骤中要清洗该数据集。

In [4]:
# merge datasets
df = pd.merge(messages,categories,on='id',how='inner')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 分割 `categories` 
- 将 `categories` 列的值根据字符 `;` 进行分割，每个值是一个新列。你会发现 [这个方法](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) 非常有用。确保设置 `expand=True`。
- 使用 categories 数据框的第一行来创建类别数据的列名。
- 使用新的列名重命名 `categories` 的列。

In [5]:
# create a dataframe of the 36 individual category columns
categories = df ['categories'].str.split (pat = ';', expand = True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [6]:
# select the first row of the categories dataframe
row = categories.iloc [0]
# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.transform(lambda x: x[:-2]).tolist()
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [7]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 转换类别值至数值 0 或 1
- 遍历 df 的类别列，只保留每个字符串的最后一个字符（1 或 0）。例如，`related-0` 变成 `0`，`related-1` 变成 `1`。将字符串转换成数值。
- 通过在 Series 后加上`.str`，你可以实现 [Pandas Series 上的常见字符串操作](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str)，如索引。你可能需要先将 Series 转换成类型字符串，你可以通过 `astype(str)` 实现。

In [8]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].transform(lambda x: x[-1:])
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric (categories[column])
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 替换 `df` `categories` 类别列
- 从 df 数据框中删除类别列，因为不再需要类别列了。
- 将 df 和 categories 数据框连接起来。

In [9]:
# drop the original categories column from `df`
df.drop (['categories'], axis = 1, inplace = True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [10]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat ([df, categories], axis = 1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 删除重复行
- 检查这个数据集中有多少重复行。
- 删除重复行。
- 确认重复行被删除了。

In [11]:
# check number of duplicates
ids = df ['message']
df [ids.isin (ids [ids.duplicated ()])].sort_values (by = ['id']).head (n=8)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
9216,10286,Me and my family every time it rains you do no...,Mwen men ak fanmil chak fwa lapli tonbe tet no...,direct,1,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
9215,10286,Me and my family every time it rains you do no...,Mwen men ak fanmil chak fwa lapli tonbe tet no...,direct,1,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
9214,10286,Me and my family every time it rains you do no...,Mwen men ak fanmil chak fwa lapli tonbe tet no...,direct,1,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
9213,10286,Me and my family every time it rains you do no...,Mwen men ak fanmil chak fwa lapli tonbe tet no...,direct,1,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,1
10158,11503,HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...,NaN,social,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
10155,11503,HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...,NaN,social,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
10156,11503,HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...,NaN,social,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
10157,11503,HELP THE EARTHQUAKE VICTIMS IN HAITI http tiny...,NaN,social,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0


In [12]:
# drop duplicates
df_new = df.drop_duplicates (subset = ['message'])
df_new.shape

(26177, 40)

In [13]:
# check number of duplicates
df_new.nunique()

id                        26177
message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_in

### 保存整理后的数据集为 sqlite databse 
你可以使用 pandas 的 [`to_sql` 方法](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) 和SQLAlchemy 库实现这一点。记得在 notebook 的第一个单元格导入 SQLAlchemy 的 `create_engine` 以在下方使用该库。

In [14]:
engine = create_engine('sqlite:///DisasterData.db')
df.to_sql('DisasterData', engine, index=False, if_exists='replace')

### 在这个notebook 里完成 `etl_pipeline.py`
使用 Resources 文件夹中给出的模板文件编写脚本，运行上面的步骤，根据用户指定的新数据库创建一个数据库。或者，你也可以在项目工作环境 IDE 中完成教室中的 `etl_pipeline.py` 。